In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from scipy import stats
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
import scipy
from scipy import stats
from scipy import spatial

In [2]:
path_text = "W2V_150.txt"
antonym_path = "Antonym_vietnamese.txt"
synonym_path = "Synonym_vietnamese.txt"

In [3]:
w2v_file = open(path_text, encoding='utf8')
antonym_file = open(antonym_path, encoding='utf8')
synonym_file = open(synonym_path, encoding='utf8')

In [4]:
words = [line for line in w2v_file.readlines()]
antonym_lines = [line for line in antonym_file.readlines()]
synonym_lines = [line for line in synonym_file.readlines()]

In [5]:
num_of_words = words[0]
vector_dimensions = words[1]
words = words[2:]

In [6]:
arr_words = []
embedding = {}

In [7]:
for idx, w in enumerate(words):
    arr_words = words[idx].split(' ')
    word = arr_words[0]
    weight = []
    for i in arr_words[2:len(arr_words)-1]:
        weight.append(float(i))
    embedding[word] = np.array(weight)

In [8]:
def generate_floats(start, end, size):
    mid = start + (end-start)/2
    x = mid + np.random.rand(size) * (end-mid)
    sign = np.random.rand(size)
    sign[sign < 0.5] = -1
    sign[sign != -1] = 1
    x = x*sign
    return x

In [9]:
generate_floats(-2, 2, 150)

array([ 0.95241035,  1.72890687, -1.24102099, -1.606396  ,  1.45401335,
        1.74104869, -0.37976942,  1.26168145,  0.34163943, -0.91179381,
       -0.89672181, -0.34666418,  0.34734914,  1.47996769, -0.11181994,
       -1.53517038, -0.48014476, -1.33112721,  0.11632293,  1.79555341,
        1.48620722, -1.98800682, -0.01066458, -1.95380099,  0.29132906,
       -0.20952752, -1.605953  ,  0.9330842 , -0.68669517,  0.83886817,
        0.50857228, -0.972421  , -1.00065165, -1.44926315,  1.69431979,
        1.75211712, -1.90137574,  1.29057611,  1.92342313,  0.11710734,
        1.90146789,  0.39975585, -1.95560364,  0.56010451, -0.22647166,
       -0.75814545,  1.75724926,  0.05925765, -0.63822265,  0.47498931,
       -0.61094165,  1.62607219,  0.9432371 ,  0.66738467, -0.54383898,
        0.72001594,  0.05673099, -0.24765063,  1.17573799, -0.56283261,
        1.32452732,  0.80622269, -1.97938518, -1.27920556, -1.94086255,
       -0.90026438, -1.04407522,  0.44949879,  0.39758785,  0.66

In [10]:
antonym_word1 = []
antonym_word2 = []
synonym_word1 = []
synonym_word2 = []

for idx, w in enumerate(antonym_lines):
    arr_words = antonym_lines[idx].split(' ')
    antonym_word1.append(arr_words[0])
    antonym_word2.append(arr_words[1].split('\n')[0])

for idx, w in enumerate(synonym_lines):
    arr_words = synonym_lines[idx].split(' ')
    synonym_word1.append(arr_words[0])
    synonym_word2.append(arr_words[1].split('\n')[0])

In [11]:
a = np.array([])
b = np.array([])

In [12]:
for i in range(len(antonym_word1)):
    if antonym_word1[i] not in embedding:
        embedding[antonym_word1[i]] = generate_floats(-2, 2, 150)
    if antonym_word2[i] not in embedding:
        embedding[antonym_word2[i]] = generate_floats(-2, 2, 150)

In [13]:
for i in range(len(antonym_word1)):
    if len(a) == 0:
        a = np.append(a, embedding[antonym_word1[i]])
        b = np.append(b, embedding[antonym_word2[i]])
    else:
        a = np.vstack((a, embedding[antonym_word1[i]]))
        b = np.vstack((b, embedding[antonym_word2[i]]))

In [14]:
for i in range(len(synonym_word1)):
    if synonym_word1[i] not in embedding:
        embedding[synonym_word1[i]] = generate_floats(-2, 2, 150)
    if synonym_word2[i] not in embedding:
        embedding[synonym_word2[i]] = generate_floats(-2, 2, 150)

In [15]:
a1 = np.array([])
b1 = np.array([])

In [16]:
for i in range(len(synonym_word1)):
    if len(a1) == 0:
        a1 = np.append(a1, embedding[synonym_word1[i]])
        b1 = np.append(b1, embedding[synonym_word2[i]])
    else:
        a1 = np.vstack((a1, embedding[synonym_word1[i]]))
        b1 = np.vstack((b1, embedding[synonym_word2[i]]))

In [17]:
y_syn = np.ones(len(a1))
y_ant = np.zeros(len(a))

In [18]:
y_train = np.concatenate((y_syn, y_ant))

In [19]:
ant_train = np.concatenate((a, b), axis=1)
syn_train = np.concatenate((a1, b1), axis=1)

In [20]:
x_train = np.concatenate((syn_train, ant_train))

In [21]:
noun_path = "ViData/ViCon/400_noun_pairs.txt"
adj_path = "ViData/ViCon/600_adj_pairs.txt"
verb_path = "ViData/ViCon/400_verb_pairs.txt"

In [22]:
noun_file = open(noun_path, encoding='utf8')
adj_file = open(adj_path, encoding='utf8')
verb_file = open(verb_path, encoding='utf8')

In [23]:
noun_line_test = [line for line in noun_file.readlines()]
verb_line_test = [line for line in verb_file.readlines()]
adj_line_test = [line for line in adj_file.readlines()]

In [24]:
noun_line_test = noun_line_test[1:]
verb_line_test = verb_line_test[1:]
adj_line_test = adj_line_test[1:]

In [25]:
words_test_1 = []
words_test_2 = []
label = []

for idx, w in enumerate(noun_line_test):
    words_test_1.append(noun_line_test[idx].split('\t')[0])
    words_test_2.append(noun_line_test[idx].split('\t')[1])
    if noun_line_test[idx].split('\t')[2].split('\n')[0] == 'ANT':
        label.append(0)
    else:
        label.append(1)
        
for idx, w in enumerate(verb_line_test):
    words_test_1.append(verb_line_test[idx].split('\t')[0])
    words_test_2.append(verb_line_test[idx].split('\t')[1])
    if verb_line_test[idx].split('\t')[2].split('\n')[0] == 'ANT':
        label.append(0)
    else:
        label.append(1)
        
for idx, w in enumerate(adj_line_test):
    words_test_1.append(adj_line_test[idx].split('\t')[0])
    words_test_2.append(adj_line_test[idx].split('\t')[1])
    if adj_line_test[idx].split('\t')[2].split('\n')[0] == 'ANT':
        label.append(0)
    else:
        label.append(1)

In [26]:
words_test_vt1 = np.array([])
words_test_vt2 = np.array([])

In [27]:
for i in range(len(words_test_1)):
    if words_test_1[i] not in embedding:
        embedding[words_test_1[i]] = generate_floats(-2, 2, 150)
    if words_test_2[i] not in embedding:
        embedding[words_test_2[i]] = generate_floats(-2, 2, 150)

In [28]:
for i in range(len(words_test_1)):
    if len(words_test_vt1) == 0:
        words_test_vt1 = np.append(words_test_vt1, embedding[words_test_1[i]])
        words_test_vt2 = np.append(words_test_vt2, embedding[words_test_2[i]])
    else:
        words_test_vt1 = np.vstack((words_test_vt1, embedding[words_test_1[i]]))
        words_test_vt2 = np.vstack((words_test_vt2, embedding[words_test_2[i]]))


In [29]:
test_data = np.concatenate((words_test_vt1, words_test_vt2), axis=1)

In [30]:
label_ = np.array(label)

In [31]:
lg = LogisticRegression(random_state=0, multi_class='ovr').fit(x_train, y_train)

In [32]:
mlp = MLPClassifier(random_state=0, learning_rate_init=0.0005).fit(x_train, y_train)

In [33]:
mlp_pred = mlp.predict(test_data)

In [34]:
lg_pred = lg.predict(test_data)

In [35]:
len(label_)

1400

In [36]:
c_lg = 0
c_mlp = 0
for i in range(len(label_)):
    if lg_pred[i] == label_[i]:
        c_lg += 1
    if mlp_pred[i] == label_[i]:
        c_mlp += 1

In [37]:
print(c_lg/len(label_))
print(c_mlp/len(label_))

0.65
0.9992857142857143


In [38]:
true_positive = 0
false_positive = 0
false_negative = 0
for i in range(len(lg_pred)):
    if lg_pred[i] == label_[i]:
        if lg_pred[i] == 1:
            true_positive += 1
    else:
        if label_[i] == 1:
            false_positive += 1
        else:
            false_negative += 1

In [39]:
precision = (true_positive) / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
F1 = 2 * ((precision * recall) / (precision + recall))

In [40]:
print(precision)
print(recall)
print(F1)

0.9371428571428572
0.5952813067150635
0.7280799112097669


In [41]:
def __test__(w1, w2):
    x = np.concatenate((embedding[w1], embedding[w2]), axis=0)
    x = x.reshape(1, 300)
    return mlp.predict(x)

In [42]:
__test__('bao_vây', 'giải_vây')

array([0.])